# How to loop / iterate over rows in pandas DataFrame
Check out the YouTube video that explains this topic: https://youtu.be/CG3EV7UBELA

In [13]:
import pandas as pd
import numpy as np

Create a dataset with 10,000 rows of random values between 0 and 1

In [2]:
df = pd.DataFrame(np.random.rand(10000,5), columns=('A','B','C','D','E'))

In [3]:
df.describe()

,A,B,C,D,E
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.495875,0.501159,0.505672,0.498235,0.500371
std,0.287760,0.288535,0.288424,0.289690,0.288282
min,0.000085,0.000168,0.000319,0.000061,0.000745
25%,0.245046,0.248718,0.256635,0.247238,0.251909
50%,0.495213,0.502304,0.503737,0.497515,0.501265
75%,0.747351,0.748981,0.758579,0.749634,0.749646
max,0.999935,0.999993,0.999920,0.999729,0.999960


## iterrow

The iterrow method is probably the most intuiative method for iterating over a DataFrame (df). It will return the index and the row of the df. <br>
Below we create a function that loops over each row of the df and replaces values that are less than 0.5 as 0 and everything else as 1. <br>
i = the index position <br>
row = the row data contained at current index location <br>
You can also try .itertuples() which is slightly faster than iterrow. I'm not going to show it in this project. 

First let's see exactly what i and row returns. Let's look at the first 5 iterations. 

In [4]:
for i, row in df.iterrows():
    while i < 3:
        print(f'i = {i}')
        print('')
        print('Returned Row:')
        print(row)
        print('')
        print(f"Column B: {row['B']}")
        print("<----------------------------------->")
        print('')
        break       

i = 0

Returned Row:
A    0.532558
B    0.247751
C    0.346707
D    0.738579
E    0.746897
Name: 0, dtype: float64

Column B: 0.24775090146495138
<----------------------------------->

i = 1

Returned Row:
A    0.524584
B    0.817626
C    0.112106
D    0.983873
E    0.294330
Name: 1, dtype: float64

Column B: 0.8176263503638649
<----------------------------------->

i = 2

Returned Row:
A    0.762473
B    0.718644
C    0.976873
D    0.043585
E    0.080178
Name: 2, dtype: float64

Column B: 0.7186444081269067
<----------------------------------->



If we compare the first three rows returned with df.iterrows to the first three rows from df.head(3) we see they are the same :)

In [5]:
df.head(3)

,A,B,C,D,E
0,0.532558,0.247751,0.346707,0.738579,0.746897
1,0.524584,0.817626,0.112106,0.983873,0.294330
2,0.762473,0.718644,0.976873,0.043585,0.080178


In [6]:
def iterrow_example(df, col):
    for i, row in df.iterrows():
        val = row[col]
        if val < .5:
            df.at[i,col] = 0     
        else:
            df.at[i,col] = 1    

Here we use the magic method timeit to let us know how long it takes to execute the line of code <br>
We allso call the iterrow_example function <br>
In this example I got 553 ms ± 8.24 ms (If you execute this code it may be different for you, but the patterns bellow should hold)

In [7]:
%timeit iterrow_example(df, 'A')

423 ms ± 23.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## pd.loc[] / pd.iloc[]

Another way we can loop over the rows of a df is to use pd.loc or pd.iloc. <br>
We create a function below to iterate over the df and then we call it. 

In [8]:
def iloc_test(df, col):
    for i in df.index:
        val = df[col].iloc[i]
        if val < .5:
            df.at[i,col] = 0     
        else:
            df.at[i,col] = 1    

In this example I got a time of 178 ms ± 1.65 ms. 3 times faster. Nice :), but wait... we can do better. 

In [9]:
%timeit iloc_test(df, 'B')

161 ms ± 19.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## apply with lambda

Here we use the .apply method with a lambda function to get even better performance. <br>
In this example I got 3.51 ms ± 216 µs. <br>
That's over 50 times faster than our previous best with pd.iloc! <br>
LET'S GO FASTER!!!

In [10]:
%timeit df['C'] = df['C'].apply(lambda x : 0 if x < .5 else 1)

2.54 ms ± 46.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## np.where()

https://www.youtube.com/watch?v=nxWginnBklU

np.where(
    conditional statment -> bool array, 
    series/array/function()/scalar if true,
    series/array/function()/scalar if False
)

Here we use the np.where method where we pass in our entire column of data with the condition we are testing for. Then we pass in the value we want if the condition is true and the value if the condition is false. 

223 µs ± 13.4 µs! 16 times faster than our previous best with .apply() <br>
Can we go any faster? Yup!

In [11]:
%timeit df['D'] = np.where(df['D'] < .5, 0, 1)

180 µs ± 3.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


When can squeeze out a little more performance by converting our pandas series into a numpy array by using .values

93.2 µs ± 527 ns <br>
Over 2 times speed up from our previous best and almost 6,000 times faster than df.iterrows. <br>
These numbers are relative though. With a smaller amount of data the differences wouldn't be as big, but with larger datasets the differences would be even more exaggerated. 

In [12]:
%timeit df['E'] = np.where(df['E'].values < .5, 0, 1)

97.8 µs ± 1.96 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


If you have elif conditions thne use numpy.select instead. <br>
conditions can be chained ((df[col ==no]) & (df['Providers']==0)) <br>

```python
conditions = [] # List all the conditions 
choices = [] # List all of the options
df[col] = np.select(conditions, choices, default='NA') 
```

 

## References

**YouTube Videos (Pycon Talks)** <br>
1000x faster data manipulation: vectorizing with Pandas and Numpy <br>
https://www.youtube.com/watch?v=nxWginnBklU

Sofia Heisler No more sad pandas optimizing pandas code for speed and efficiency PyCon 2017 <br>
https://www.youtube.com/watch?v=HN5d490_KKk&feature=youtu.be <br>
<br>
<br>
**Articles** <br>
Optimum approach for iterating over a DataFrame <br>
https://medium.com/@rtjeannier/pandas-101-cont-9d061cb73bfc

Crude looping in Pandas, or That Thing You Should Never Ever Do <br>
https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6 
